In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta
import math

In [106]:
times02 = pd.read_csv("./feature_classifier_results02.txt")
times01 = pd.read_csv("./feature_classifier_results01.txt")
times02 = times02.dropna(subset=["end_time"])

times = pd.concat([times02, times01])
times

times = times.astype({"test_time":"int64", "end_time":"int64"})

times[["Prop", "Min", "Splitter_Seed"]] = times["splitter"].str.split("/", expand=True)
times

times[["Splitter","Seed"]] = times["Splitter_Seed"].str.split("_", expand=True)
times

times = times.drop(index = times.loc[(times["Prop"] == "prop_0-05") & (times["Min"] == "min_5")].index)
times

times["Train_Time"] = times["test_time"] - times["start_time"]
times["Test_Time"] = times["end_time"] - times["test_time"]
times["Total_Time"] = times["end_time"] - times["start_time"]


times = times.sort_values(by=["Total_Time"])
times

,splitter,level,start_time,test_time,end_time,Prop,Min,Splitter_Seed,Splitter,Seed,Train_Time,Test_Time,Total_Time
109,prop_0-1/min_10/RandomSplit_101,species,1738608130,1738608170,1738608191,prop_0-1,min_10,RandomSplit_101,RandomSplit,101,40,21,61
134,prop_0-1/min_10/StratifiedSplit2_56,species,1738611295,1738611335,1738611356,prop_0-1,min_10,StratifiedSplit2_56,StratifiedSplit2,56,40,21,61
139,prop_0-1/min_10/StratifiedSplit2_84,species,1738611928,1738611968,1738611989,prop_0-1,min_10,StratifiedSplit2_84,StratifiedSplit2,84,40,21,61
154,prop_0-1/min_10/StratifiedSplit2_105,species,1738613823,1738613863,1738613884,prop_0-1,min_10,StratifiedSplit2_105,StratifiedSplit2,105,40,21,61
459,prop_0-05/min_10/StratifiedSplit2_84,species,1738653960,1738654001,1738654021,prop_0-05,min_10,StratifiedSplit2_84,StratifiedSplit2,84,41,20,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,prop_0-1/min_5/RandomSplit_92,genus,1738338064,1738338208,1738338241,prop_0-1,min_5,RandomSplit_92,RandomSplit,92,144,33,177
59,prop_0-1/min_5/StratifiedSplit2_105,genus,1738344417,1738344561,1738344594,prop_0-1,min_5,StratifiedSplit2_105,StratifiedSplit2,105,144,33,177
31,prop_0-1/min_5/RandomSplit_227,genus,1738339958,1738340102,1738340135,prop_0-1,min_5,RandomSplit_227,RandomSplit,227,144,33,177
51,prop_0-1/min_5/StratifiedSplit2_92,genus,1738343147,1738343292,1738343325,prop_0-1,min_5,StratifiedSplit2_92,StratifiedSplit2,92,145,33,178


In [107]:
times = times.groupby(by=["level", "Prop"]).agg({
    "Train_Time":["min","mean","max"],
    "Test_Time":["min","mean","max"],
    "Total_Time":["min","mean","max"],
}).reset_index()

times

level       Prop Train_Time                 Test_Time                \
                              min       mean  max       min      mean max   
0     class  prop_0-05        114  116.12500  118        24  24.25000  25   
1     class   prop_0-1        110  112.12500  116        28  29.03125  30   
2     class   prop_0-2        101  103.31250  109        38  38.37500  39   
3    family  prop_0-05        105  106.62500  110        24  24.18750  25   
4    family   prop_0-1        101  106.56250  115        28  28.78125  30   
5    family   prop_0-2         93   97.96875  104        36  37.46875  39   
6     genus  prop_0-05         85   86.43750   89        23  23.62500  24   
7     genus   prop_0-1         82  109.96875  145        26  29.68750  33   
8     genus   prop_0-2         77  101.37500  132        32  37.84375  44   
9     order  prop_0-05        107  107.81250  109        24  24.18750  25   
10    order   prop_0-1        103  105.31250  109        28  28.62500  30   
11    order   prop_0-2         95   97.84375  101        36  37.34375  39   
12  species  prop_0-05         40   40.31250   41        20  20.81250  21   
13  species   prop_0-1         40   48.34375   58        21  22.40625  24   
14  species   prop_0-2         38   46.46875   55        22  24.31250  27   

   Total_Time                  
          min       mean  max  
0         139  140.37500  142  
1         139  141.15625  146  
2         139  141.68750  147  
3         129  130.81250  134  
4         129  135.34375  144  
5         129  135.43750  142  
6         109  110.06250  113  
7         109  139.65625  178  
8         109  139.21875  176  
9         131  132.00000  133  
10        131  133.93750  138  
11        132  135.18750  139  
12         61   61.12500   62  
13         61   70.75000   81  
14         61   70.78125   81

In [112]:
times["Train_Time_min"] = times.loc[:,("Train_Time", "min")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Train_Time_max"] = times.loc[:,("Train_Time", "max")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Train_Time_mean"] = times.loc[:,("Train_Time", "mean")].apply(lambda x: str(timedelta(seconds=math.floor(x))))

times["Test_Time_min"] = times.loc[:,("Test_Time", "min")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Test_Time_max"] = times.loc[:,("Test_Time", "max")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Test_Time_mean"] = times.loc[:,("Test_Time", "mean")].apply(lambda x: str(timedelta(seconds=math.floor(x))))

times["Total_Time_min"] = times.loc[:,("Total_Time", "min")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Total_Time_max"] = times.loc[:,("Total_Time", "max")].apply(lambda x: str(timedelta(seconds=math.floor(x))))
times["Total_Time_mean"] = times.loc[:,("Total_Time", "mean")].apply(lambda x: str(timedelta(seconds=math.floor(x))))

times = times.sort_values(by=[("Total_Time", "min")], ascending=False)
# times["Test_Time"] = times["Test_Time"].apply(lambda x: str(timedelta(seconds=math.floor(x))))
# times["Total_Time"] = times["Total_Time"].apply(lambda x: str(timedelta(seconds=math.floor(x))))

In [113]:
times[
        [
            "level",
            "Prop",
            # 'Train_Time_min',
            # 'Train_Time_max',
            'Train_Time_mean',
            # 'Test_Time_min',
            # 'Test_Time_max',
            'Test_Time_mean',
            # 'Total_Time_min',
            # 'Total_Time_max',
            'Total_Time_mean'
        ]
    ]

,level,Prop,Train_Time_mean,Test_Time_mean,Total_Time_mean
,,,,,
1,class,prop_0-1,0:01:52,0:00:29,0:02:21
2,class,prop_0-2,0:01:43,0:00:38,0:02:21
0,class,prop_0-05,0:01:56,0:00:24,0:02:20
11,order,prop_0-2,0:01:37,0:00:37,0:02:15
9,order,prop_0-05,0:01:47,0:00:24,0:02:12
10,order,prop_0-1,0:01:45,0:00:28,0:02:13
3,family,prop_0-05,0:01:46,0:00:24,0:02:10
5,family,prop_0-2,0:01:37,0:00:37,0:02:15
4,family,prop_0-1,0:01:46,0:00:28,0:02:15
